In [ ]:
import pickle
import gzip
import os, sys, errno
sys.path.insert(0, '../configuration')
sys.path.insert(0, '../utils')
sys.path.insert(0,"../io_funcs")
from binary_io import BinaryIOCollection
from configuration import configuration
from utils import read_file_list, prepare_file_path_list
import time
import math

import subprocess
import logging
import numpy
#import gnumpy as gnp
# we need to explicitly import this in some cases, not sure why this doesn't get imported with numpy itself
import numpy.distutils.__config__
# and only after that can we import theano
import pdb